In [1]:
import os
import dlib
import sys
import numpy as np

import pandas as pd
import pickle
import os,os.path
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [2]:
#!pip install face_recognition

In [3]:
# dlib.DLIB_USE_CUDA=1
# dlib.USE_AVX_INSTRUCTIONS=1
import face_recognition as fr

In [9]:
path = os.getcwd() + '/'
path

'/media/omkar/B608-4190/PA/Classification/'

In [10]:
#os.listdir(path)

In [11]:
list_data = ['final_frontal0.csv',
 'final_frontal1.csv',
 'final_frontal2.csv',
 'final_frontal3.csv',
 'final_frontal4.csv']

In [12]:
data = pd.read_csv(path+'final_frontal0.csv')
data.head()

,Unnamed: 0,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,0,30601258@N03,10424815813_e94629b1ec_o.jpg,2,"(25, 32)",m,301,105,640,641,0,0,94
1,1,30601258@N03,10437979845_5985be4b26_o.jpg,3,"(25, 32)",m,752,1255,484,485,180,0,47
2,2,30601258@N03,11816644924_075c3d8d59_o.jpg,2,"(25, 32)",m,175,80,769,768,-75,0,34
3,3,30601258@N03,10424595844_1009c687e4_o.jpg,4,"(38, 43)",f,1912,905,1224,1224,155,0,64
4,4,30601258@N03,10190308156_5c748ab2da_o.jpg,2,"(25, 32)",m,1013,1039,453,452,-75,0,59


In [13]:
data = data[['user_id' ,'original_image',  'gender' ,'age']]
data.head()

,user_id,original_image,gender,age
0,30601258@N03,10424815813_e94629b1ec_o.jpg,m,"(25, 32)"
1,30601258@N03,10437979845_5985be4b26_o.jpg,m,"(25, 32)"
2,30601258@N03,11816644924_075c3d8d59_o.jpg,m,"(25, 32)"
3,30601258@N03,10424595844_1009c687e4_o.jpg,f,"(38, 43)"
4,30601258@N03,10190308156_5c748ab2da_o.jpg,m,"(25, 32)"


In [0]:
def feature_extract(data):
    file_path = path + 'file/'
    count = 0
    for index,row in data.iterrows():
        if os.path.isfile(file_path + row['user_id'] +'/' + row['original_image']):
            encode = fr.face_encodings(fr.load_image_file(file_path + row['user_id'] +'/' + row['original_image']))
            if len(encode)>=1:
                encode = encode[0]
            else:
                continue
            if count == 0:
                temp = np.array([row['user_id'],row['original_image'],row['gender'],row['age']])
                temp = np.append(temp,encode)
                df = pd.DataFrame(temp).T
                count +=1
            else:
                temp = np.array([row['user_id'],row['original_image'],row['gender'],row['age']])
                temp = np.append(temp,encode)
                df = df.append(pd.DataFrame(temp).T ,ignore_index=True)           
    return df

In [0]:
data.shape

(3210, 4)

In [0]:
final_df = feature_extract(data)

In [0]:
final_df.to_csv(path + 'feature0.csv')

In [0]:
for i in range(len(list_data)):
    #print(list_data[i])
    data = pd.read_csv(path+ list_data[i])
    data = data[['user_id' ,'original_image',  'gender' ,'age']]
    temp_df = feature_extract(data)
    #print(temp_df.shape)
    temp_df.to_csv(path + 'feature' + str(i) + '.csv')
    if i==0:
        final_df = temp_df.copy()
    else:
        final_df = final_df.append(temp_df,ignore_index=True)
    #print(final_df.shape)

In [0]:
final_df.shape

(10352, 265)

In [0]:
del final_df

In [14]:
for i in range(len(list_data)):
    read_df = pd.read_csv(path + 'feature' + str(i)+'.csv')
    if i==0:
        final_df = pd.DataFrame(read_df)
    else:
        final_df = final_df.append(read_df,ignore_index=True)
    

In [15]:
final_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,122,123,124,125,126,127,128,129,130,131
0,0,30601258@N03,10424815813_e94629b1ec_o.jpg,m,"(25, 32)",-0.083294,0.087730,0.054649,-0.078772,-0.080640,...,0.077006,0.031769,-0.011822,-0.080471,-0.171120,-0.138578,-0.046703,0.034158,0.044308,0.026031
1,1,30601258@N03,10437979845_5985be4b26_o.jpg,m,"(25, 32)",-0.164970,0.121995,0.052119,-0.091033,-0.089395,...,0.089980,0.020260,0.035951,-0.019216,-0.122825,-0.077148,0.033474,0.007737,0.004064,0.013872
2,2,30601258@N03,11816644924_075c3d8d59_o.jpg,m,"(25, 32)",-0.115149,0.089949,0.051598,-0.037451,-0.072356,...,0.027709,0.034753,-0.023607,-0.066158,-0.171793,-0.112601,-0.049470,-0.017822,0.052654,0.060677
3,3,30601258@N03,10424595844_1009c687e4_o.jpg,f,"(38, 43)",-0.112837,0.001386,-0.030106,-0.068910,-0.092794,...,-0.044121,0.028554,-0.055196,-0.032632,-0.267007,-0.110302,-0.023602,-0.005010,0.025858,0.025436
4,4,30601258@N03,10190308156_5c748ab2da_o.jpg,m,"(25, 32)",-0.101052,0.093247,0.077053,-0.071299,-0.090539,...,0.003219,0.025305,-0.047061,-0.062078,-0.204218,-0.132275,-0.058921,0.034308,0.060252,0.074107


In [16]:
trial_df = final_df.copy()
trial_df.shape

(12545, 133)

In [17]:
img_df = trial_df[['0' , '1' , '2' , '3']]

In [18]:
trial_df = trial_df.drop(['0','1','2','3'],axis=1)

In [19]:
trial_df = trial_df.drop('Unnamed: 0',axis=1)

In [20]:
trial_df.shape

(12545, 128)

In [21]:
img_df.head()

,0,1,2,3
0,30601258@N03,10424815813_e94629b1ec_o.jpg,m,"(25, 32)"
1,30601258@N03,10437979845_5985be4b26_o.jpg,m,"(25, 32)"
2,30601258@N03,11816644924_075c3d8d59_o.jpg,m,"(25, 32)"
3,30601258@N03,10424595844_1009c687e4_o.jpg,f,"(38, 43)"
4,30601258@N03,10190308156_5c748ab2da_o.jpg,m,"(25, 32)"


In [22]:
y = img_df.copy()

In [23]:
y = y['2']

In [24]:
y.unique()

array(['m', 'f', nan, 'u'], dtype=object)

In [25]:
y.shape

(12545,)

In [26]:
from random import randint
y = y.fillna(randint(0,2))

In [27]:
y = y.replace('f',0)
y = y.replace('m',1)
y = y.replace('u',2)

In [28]:

y.unique()

array([1, 0, 2])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(trial_df, y,test_size=0.3, random_state=17)

In [30]:
X_train.shape , y_train.shape

((8781, 128), (8781,))

In [31]:
logreg = LogisticRegression(random_state=17,multi_class='auto')
logreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

/home/omkar/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic regression classifier on training set: 0.76
Accuracy of Logistic regression classifier on test set: 0.74


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 0.65


In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Accuracy of K-NN classifier on training set: 0.71
Accuracy of K-NN classifier on test set: 0.55


In [0]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))

Accuracy of GNB classifier on training set: 0.66
Accuracy of GNB classifier on test set: 0.64


In [0]:
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of SVM classifier on training set: 0.53
Accuracy of SVM classifier on test set: 0.51


In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = logreg.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[1394  372   17]
 [ 485 1282    6]
 [  47  102   59]]
              precision    recall  f1-score   support

         0.0       0.72      0.78      0.75      1783
         1.0       0.73      0.72      0.73      1773
         2.0       0.72      0.28      0.41       208

   micro avg       0.73      0.73      0.73      3764
   macro avg       0.72      0.60      0.63      3764
weighted avg       0.73      0.73      0.72      3764



In [0]:
penalty = ['l1', 'l2']
para_C = [0.001, 0.01, 0.1, 1, 10, 100, 1000] 
hyperparameters = dict(C=para_C, penalty=penalty)
# para_C

[0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [0]:
logreg = LogisticRegression(random_state=17,multi_class='auto')
clf = GridSearchCV(logreg, hyperparameters, cv=5, verbose=0)

In [0]:
best_model = clf.fit(X_train , y_train)

In [0]:
best_model.best_score_

0.7561781118323654

In [0]:
best_model.best_estimator_.fit(X_train,y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'.format(best_model.best_estimator_.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'.format(best_model.best_estimator_.score(X_test, y_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic regression classifier on training set: 0.76
Accuracy of Logistic regression classifier on test set: 0.75


In [33]:
model_file = path + 'model.sav'

In [34]:
pickle.dump(logreg,open(model_file,'wb'))

In [0]:
model = pickle.load(open(model_file,'rb'))